In [93]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"




# Spark ML Selección de Variables

Cargamos un dataset con información sobre partos. Este dataset tiene como variable objetivo el fallecimiento o supervivencia de los recién nacidos.



### Crear SparkSession

In [94]:
# Respuesta

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



### Cargar datos y comprobar schema

In [95]:
# Respuesta

births = spark.read.csv(DATA_PATH+'births_train.csv',sep=',', header=True, inferSchema=True) # You may try with: births.csv

births.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: string (nullable = true)
 |-- BIRTH_YEAR: integer (nullable = true)
 |-- BIRTH_MONTH: integer (nullable = true)
 |-- BIRTH_PLACE: integer (nullable = true)
 |-- MOTHER_AGE_YEARS: integer (nullable = true)
 |-- MOTHER_RACE_6CODE: integer (nullable = true)
 |-- MOTHER_EDUCATION: integer (nullable = true)
 |-- FATHER_COMBINED_AGE: integer (nullable = true)
 |-- FATHER_EDUCATION: integer (nullable = true)
 |-- MONTH_PRECARE_RECODE: integer (nullable = true)
 |-- CIG_BEFORE: integer (nullable = true)
 |-- CIG_1_TRI: integer (nullable = true)
 |-- CIG_2_TRI: integer (nullable = true)
 |-- CIG_3_TRI: integer (nullable = true)
 |-- MOTHER_HEIGHT_IN: integer (nullable = true)
 |-- MOTHER_BMI_RECODE: integer (nullable = true)
 |-- MOTHER_PRE_WEIGHT: integer (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: integer (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: integer (nullable = true)
 |-- DIABETES_PRE: string (nullable = true)
 |-- DIABETES_GEST: string (nullable = t

In [96]:
# Respuesta

births.show(1)

+----------------------+----------+-----------+-----------+----------------+-----------------+----------------+-------------------+----------------+--------------------+----------+---------+---------+---------+----------------+-----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+-------+----------------------+---------+----------+--------+-----------+----------+------------------------+---------------+-------+--------------+--------+---------------+----------+-------------------------+-------------------+-------------------+------------------------+---------------------+----------------+------------------+---------------+-----------------------+----------------+-----------------------+---------------------+--------------------+-------------------------------------+--------------------------------------+----------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_YEAR|BIRTH_MONTH|BIRTH_PLACE|MOTHER_AGE_

 

Se puede ver la variable objetivo (_target_) "INFANT_ALIVE_AT_REPORT" es de tipo "string".  Para realizar los siguientes análisis la misma debe ser convertida a tipo numérico.

In [97]:
# Respuesta

from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

births= births.withColumn("INFANT_ALIVE_AT_REPORT", F.udf(lambda x: 1.0 if x == "Y" else 0.0, FloatType())("INFANT_ALIVE_AT_REPORT") )

births.groupby("INFANT_ALIVE_AT_REPORT").count().show()

+----------------------+-----+
|INFANT_ALIVE_AT_REPORT|count|
+----------------------+-----+
|                   1.0|23349|
|                   0.0|22080|
+----------------------+-----+





Todas las variables de tipo 'string' son categóricas. Hagamos un StringIndexer con ellas

In [98]:
# Respuesta

from pyspark.ml.feature import StringIndexer

stringindexer_dictionary ={}




Es una buena idea guardar todos los modelos para poder acceder a los cambios para cada categoría

In [99]:
# Respuesta

for element in births.dtypes:
    if element[1] == 'string':
        stringindexer = StringIndexer(inputCol = element[0], outputCol=element[0]+'_indexed')
        stringindexer_model = stringindexer.fit(births)
        births = stringindexer_model.transform(births)
        births = births.drop(element[0])
        
        stringindexer_dictionary[element[0]] = stringindexer_model



### ChiSquared

Este método de selección de variables se aplica a variables categóricas.

In [100]:
# Respuesta

from pyspark.ml.feature import ChiSqSelector, VectorAssembler



El primer paso es crear un VectorAssembler de las variables categóricas

In [101]:
# Respuesta

input_cols = [element for element in births.columns if '_indexed' in element]

vectorassembler = VectorAssembler(inputCols=input_cols, outputCol='categorical_assembled')
births = vectorassembler.transform(births)

chisquared = ChiSqSelector(featuresCol=vectorassembler.getOutputCol(), labelCol="INFANT_ALIVE_AT_REPORT", numTopFeatures=5)
chisquared_model = chisquared.fit(births)



Ver resultado:

In [102]:
# Respuesta

chisquared_model.selectedFeatures

[0, 1, 2, 3, 4]



Ver nombre de las variables:



Nota: el orden viene dado por el orden en que se han introducido las variables en el VectorAssembler.

In [105]:
# Respuesta

top_5_categorical = [input_cols[index] for index in chisquared_model.selectedFeatures]
top_5_categorical

['DIABETES_PRE_indexed',
 'DIABETES_GEST_indexed',
 'HYP_TENS_PRE_indexed',
 'HYP_TENS_GEST_indexed',
 'PREV_BIRTH_PRETERM_indexed']



### RandomForest

Este método toma tanto variables categóricas como numéricas. Tiene el incoveniente que es aleatorio (_random_) y los resultados pueden verse modificados. Vamos a ver como contrarrestar este problema. Además, retorna importancia de las variables.



Importamos RandomForestClassifier puesto que se trata de un problema de clasificación

In [37]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier



Debemos volver a crear un VectorAssembler y ejecutamos el algoritmo RandomForestClassifier.

In [38]:
# Respuesta

features_for_rf = [element for element in births.columns if element != 'INFANT_ALIVE_AT_REPORT' and element !='categorical_assembled']

In [39]:
# Respuesta

vectorassembler = VectorAssembler(inputCols=features_for_rf, outputCol='assembled_rf')
births = vectorassembler.transform(births)

rf = RandomForestClassifier(featuresCol=vectorassembler.getOutputCol(), labelCol='INFANT_ALIVE_AT_REPORT')

rf_model = rf.fit(births)



De nuevo, el índice del vector viene dado por el orden de entrada de las variables en el VectorAssembler

In [40]:
# Respuesta

rf_model.featureImportances

SparseVector(53, {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0001, 4: 0.0, 5: 0.0002, 6: 0.0001, 7: 0.005, 8: 0.0038, 9: 0.0001, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0017, 15: 0.0001, 16: 0.0, 17: 0.0097, 18: 0.0, 19: 0.0, 20: 0.0017, 21: 0.0011, 22: 0.2194, 23: 0.1516, 24: 0.1991, 25: 0.0527, 26: 0.1355, 27: 0.1296, 28: 0.0182, 29: 0.0057, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0001, 34: 0.0, 35: 0.0, 36: 0.0005, 37: 0.0, 39: 0.001, 40: 0.0, 41: 0.0027, 42: 0.0052, 43: 0.0102, 44: 0.0022, 45: 0.0002, 46: 0.0, 47: 0.0003, 49: 0.0, 50: 0.0, 51: 0.0002, 52: 0.0417})



* Ordenamos importancias para quedarnos con las variables que explican el x% de importancia respecto la variable objetivo. Tomaremos el 95% de importancia



Primero, creamos una lista y almacenamos el índice para poder recuperar el nombre de la variable más adelante


In [41]:
# Respuesta

importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]



Ordenamos de mayor a menor importancia

In [42]:
# Respuesta

importances = sorted(importances, key=lambda value: value[1], reverse=True)



Nos quedamos con aquellas que explican el 95% de las variables

In [43]:
# Respuesta

compt = 0
important_features =[]
for element in importances:
    if compt < 0.95:
        compt += element[1]
        important_features.append((features_for_rf[element[0]], element[1]))

In [44]:
# Respuesta

important_features

[('APGAR_5', 0.21941245244780555),
 ('APGAR_10', 0.1990509570431592),
 ('APGAR_5_RECODE', 0.15159174784368368),
 ('OBSTETRIC_GESTATION_WEEKS', 0.13550571587277377),
 ('INFANT_WEIGHT_GRAMS', 0.1296323667129227),
 ('APGAR_10_RECODE', 0.05274068115563527),
 ('INFANT_BREASTFED_indexed', 0.04174235857620831),
 ('INFANT_NO_ABNORMALITIES', 0.018203302052444317),
 ('INFANT_NICU_ADMISSION_indexed', 0.010209149067515246)]



Ahora, veamos como evitar la aleatoriedad del RandomForest haciendo varias iteraciones y quedándonos con las variables que aparecen en todas las iteraciones



Creemos una semilla (_seed_), al inicializarlo será replicable.

In [106]:
# Respuesta

random_seed = 4
num_iter = 10

import random

In [107]:
# Respuesta

random.seed(random_seed)

random_seeds=[]

while len(set(random_seeds)) < num_iter:
    random_seeds.append(random.randint(0,10000))

features_random_seed = []
for random_seed in random_seeds:
    rf = RandomForestClassifier(featuresCol=vectorassembler.getOutputCol(), labelCol='INFANT_ALIVE_AT_REPORT', seed = random_seed )
    rf_model = rf.fit(births)
    
    importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]

    #order from highest to lowest importance
    importances = sorted(importances, key=lambda value: value[1], reverse=True)

    #select those that explain 95% of the target variable

    compt = 0
    important_features =[]
    for element in importances:
        if compt < 0.95:
            compt += element[1]
            important_features.append((features_for_rf[element[0]], element[1]))
    features_random_seed.append(important_features)


In [47]:
# Respuesta

features_random_seed

[[('OBSTETRIC_GESTATION_WEEKS', 0.1897563919934772),
  ('APGAR_5_RECODE', 0.1816585782356116),
  ('APGAR_10', 0.14205212924330746),
  ('APGAR_5', 0.1178229597974382),
  ('INFANT_BREASTFED_indexed', 0.11678964589503937),
  ('APGAR_10_RECODE', 0.09949067380070632),
  ('INFANT_WEIGHT_GRAMS', 0.06964752921996402),
  ('MOTHER_WEIGHT_GAIN', 0.03395813029859042)],
 [('OBSTETRIC_GESTATION_WEEKS', 0.22373268121473489),
  ('APGAR_10', 0.17550243605514046),
  ('APGAR_5', 0.1459829268718799),
  ('APGAR_5_RECODE', 0.1413768840867261),
  ('INFANT_BREASTFED_indexed', 0.12133793312942176),
  ('INFANT_WEIGHT_GRAMS', 0.07169498002361721),
  ('APGAR_10_RECODE', 0.046964044119101676),
  ('INFANT_ASSIST_VENTI_indexed', 0.011928566811532561),
  ('INFANT_NICU_ADMISSION_indexed', 0.010463668851627677),
  ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', 0.009733998844637624)],
 [('APGAR_5_RECODE', 0.2538788456669687),
  ('APGAR_5', 0.18205609995085512),
  ('OBSTETRIC_GESTATION_WEEKS', 0.14363221561476974),
  ('APGAR



Tras construir esta lista de listas con información de cada iteración, nos quedamos con aquellas variables que aparecen en cada iteración. Veamos cómo se hace.



Primero convertimos la lista de listas en una sola lista

In [48]:
# Respuesta

flat_features = [feature for one_seed in features_random_seed for feature in one_seed]
features = [element[0] for element in flat_features]

from collections import Counter

features_all_seeds = [element[0] for element in Counter(features).items() if element[1] == num_iter]
features_all_seeds

['OBSTETRIC_GESTATION_WEEKS',
 'APGAR_10',
 'APGAR_5_RECODE',
 'APGAR_5',
 'INFANT_WEIGHT_GRAMS',
 'INFANT_BREASTFED_indexed']



Si quisiéramos el valor de la importancia en sí, calculamos media por cada semilla:

In [49]:
# Respuesta

import numpy as np

dictionary_importances = {}

for feature in features_all_seeds:
    dictionary_importances[feature] = []
    for values in features_random_seed:
        for element in values:
            if element[0] == feature:
                dictionary_importances[feature].append(element[1])
                break
    dictionary_importances[feature] = np.mean(dictionary_importances[feature])

In [50]:
# Respuesta

sorted(dictionary_importances.items(), key=lambda value: value[1], reverse=True)

[('APGAR_5_RECODE', 0.18264939681654854),
 ('APGAR_5', 0.1803826606406311),
 ('OBSTETRIC_GESTATION_WEEKS', 0.17302570129366404),
 ('APGAR_10', 0.1307703119045178),
 ('INFANT_WEIGHT_GRAMS', 0.1030116177462312),
 ('INFANT_BREASTFED_indexed', 0.0767927600327975)]